In [1]:
%gui qt


In [ ]:
import numpy as np
from PyQt5.QtWidgets import QMainWindow
from PyQt5.QtCore import QTimer
import pyqtgraph.opengl as gl
from skyfield.api import load

class PlanetTrackerGL(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Planet Tracker with PyQtGraph")
        self.resize(1000, 800)

        self.view = gl.GLViewWidget()
        self.view.setCameraPosition(distance=50)
        self.setCentralWidget(self.view)

        self.planets = load('de421.bsp')
        self.ts = load.timescale()
        self.planet_names = [
            'mercury',
            'venus',
            'earth',
            'mars',
            'jupiter barycenter',
            'saturn barycenter'
        ]

        self.planet_targets = [self.planets[name] for name in self.planet_names]
        self.planet_colors = [(1, 0.6, 0.0, 1), (1, 1, 0, 1), (0, 0.5, 1, 1), (1, 0.3, 0.3, 1), (0.7, 0.3, 1, 1), (0.9, 0.9, 0.5, 1)]
        self.planet_spheres = []

        # Starfield background - many small white points
        stars_pos = np.random.uniform(-200, 200, size=(3000, 3))
        stars = gl.GLScatterPlotItem(pos=stars_pos, size=0.4, color=(1, 1, 1, 0.3))
        self.view.addItem(stars)

        # Sun at origin
        self.sun = gl.GLScatterPlotItem(pos=np.array([[0, 0, 0]]), size=6, color=(1, 1, 0, 1))
        self.view.addItem(self.sun)

        # Create spheres for planets
        for color in self.planet_colors:
            sphere = gl.GLScatterPlotItem(pos=np.array([[0, 0, 0]]), size=2.5, color=color)
            self.view.addItem(sphere)
            self.planet_spheres.append(sphere)

        self.t = self.ts.now()
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_positions)
        self.timer.start(30)  # update ~33 times per second

        self.show()

    def update_positions(self):
        sun = self.planets['sun']
        self.t = self.ts.tt_jd(self.t.tt + 0.02)  # increment time slightly

        for i, target in enumerate(self.planet_targets):
            pos = sun.at(self.t).observe(target).position.au
            # Update planet sphere position
            self.planet_spheres[i].setData(pos=np.array([pos]))

# Create and show the window
tracker = PlanetTrackerGL()


In [ ]:
import sys
import numpy as np
from PyQt5.QtWidgets import QMainWindow, QLabel, QApplication
from PyQt5.QtCore import QTimer, Qt
import pyqtgraph.opengl as gl
from pyqtgraph.opengl import GLLinePlotItem
from skyfield.api import load
from PyQt5.QtGui import QVector4D

class PlanetTrackerGL(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Planet Tracker with PyQtGraph")
        self.resize(1000, 800)

        self.view = gl.GLViewWidget()
        self.view.setCameraPosition(distance=50)
        self.setCentralWidget(self.view)

        self.planets = load('de421.bsp')
        self.ts = load.timescale()
        self.planet_names = [
            'mercury',
            'venus',
            'earth',
            'mars',
            'jupiter barycenter',
            'saturn barycenter'
        ]

        self.planet_targets = [self.planets[name] for name in self.planet_names]
        self.planet_colors = [
            (1, 0.6, 0.0, 1), (1, 1, 0, 1), (0, 0.5, 1, 1),
            (1, 0.3, 0.3, 1), (0.7, 0.3, 1, 1), (0.9, 0.9, 0.5, 1)
        ]
        self.planet_spheres = []
        self.orbit_lines = []
        self.labels = []

        # Starfield background
        stars_pos = np.random.uniform(-200, 200, size=(3000, 3))
        stars = gl.GLScatterPlotItem(pos=stars_pos, size=0.4, color=(1, 1, 1, 0.3))
        self.view.addItem(stars)

        # Sun at origin
        self.sun = gl.GLScatterPlotItem(pos=np.array([[0, 0, 0]]), size=6, color=(1, 1, 0, 1))
        self.view.addItem(self.sun)

        # Create spheres for planets
        for color in self.planet_colors:
            sphere = gl.GLScatterPlotItem(pos=np.array([[0, 0, 0]]), size=2.5, color=color)
            self.view.addItem(sphere)
            self.planet_spheres.append(sphere)

        # Create orbit lines for each planet
        self.create_orbits()

        # Create labels (QLabels) for each planet
        for name, color in zip(self.planet_names, self.planet_colors):
            label = QLabel(name.capitalize(), self)
            label.setStyleSheet(f"color: rgb({int(color[0]*255)}, {int(color[1]*255)}, {int(color[2]*255)});"
                                "font-weight: bold; background: transparent;")
            label.setAttribute(Qt.WA_TransparentForMouseEvents)
            label.adjustSize()
            self.labels.append(label)

        self.t = self.ts.now()
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_positions)
        self.timer.start(30)

        self.show()

    def get_orbital_period_days(self, planet_name):
        # Approximate orbital periods in Earth days
        orbital_periods = {
            'mercury': 87.969,
            'venus': 224.701,
            'earth': 365.256,
            'mars': 686.980,
            'jupiter barycenter': 4332.59,
            'saturn barycenter': 10759.22
        }
        return orbital_periods.get(planet_name.lower(), 365.25) 

    def create_orbits(self):
        steps = 500 

        for target, color, name in zip(self.planet_targets, self.planet_colors, self.planet_names):
            orbital_period = self.get_orbital_period_days(name)
            positions = []
            t0 = self.ts.now().tt
            half_period = orbital_period / 2
            for i in range(steps):
                # From -half_period to +half_period days relative to now for full orbit coverage
                delta_days = -half_period + (i / (steps - 1)) * orbital_period
                t_future = t0 + delta_days
                time_obj = self.ts.tt_jd(t_future)
                pos = self.planets['sun'].at(time_obj).observe(target).position.au
                positions.append(pos)
            positions = np.array(positions)
            line = GLLinePlotItem(pos=positions, color=color, width=1, antialias=True)
            self.view.addItem(line)
            self.orbit_lines.append(line)

    def update_positions(self):
        sun = self.planets['sun']
        self.t = self.ts.tt_jd(self.t.tt + 0.02)

        for i, target in enumerate(self.planet_targets):
            pos = sun.at(self.t).observe(target).position.au
            self.planet_spheres[i].setData(pos=np.array([pos]))
            self.update_label_position(i, pos)

    def update_label_position(self, i, pos_3d):
        x, y, z = pos_3d

        # Project 3D position to 2D screen coords
        m = self.view.projectionMatrix()
        v = self.view.viewMatrix()

        p = QVector4D(x, y, z, 1.0)
        clip_space = m * v * p

        if clip_space.w() != 0:
            ndc = clip_space / clip_space.w()
            w, h = self.view.width(), self.view.height()
            screen_x = (ndc.x() + 1) / 2 * w
            screen_y = (1 - ndc.y()) / 2 * h

            label = self.labels[i]
            label.move(int(screen_x), int(screen_y))
            label.show()
        else:
            self.labels[i].hide()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    tracker = PlanetTrackerGL()
    sys.exit(app.exec_())
